In [1]:
pip install -q wikipedia

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: Vector database\nSummary: A vector database management system (VDBMS) or simply vector database or vector store is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In t'

# Reasoning and Acting (React)

## Creating a ReAct Agent

In [3]:
#pip install langchainhub  -q

In [4]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [17]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

template = '''
Answer the following questions in GERMAN as best uou can.
Questions: {q}
'''
prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name = 'Python REPL',
    func = python_repl.run,
    description = 'Useful when you need to use Python to answer a question. You Should input Python code.'
)

# 2. Wikipedia Tool (for searching wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on wikipedia'
)
# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool cant provide'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations= 10
)

In [11]:
question = 'Generate the first 20 numbers in the Fibonacci series'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use Python to generate the Fibonacci series.
Action: Python REPL
Action Input: 
```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        fib_series.append(fib_series[i-1] + fib_series[i-2])
    return fib_series

fibonacci(20)
```I now have the first 20 numbers in the Fibonacci series.
Final Answer: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]

> Finished chain.


In [12]:
question = 'Who is the current prime minister of the UK?'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...
I should use DuckDuckGo Search to find the current prime minister of the UK.
Action: DuckDuckGo Search
Action Input: "current prime minister of the UK"Modern historians generally consider Robert Walpole, who led the government of the Kingdom of Great Britain for over twenty years from 1721, [5] as the first prime minister. Walpole is also the longest-serving British prime minister by this definition. [6] LONDON, Nov 13 (Reuters) - David Cameron, 57, served as British prime minister from 2010 to 2016, resigning after the outcome of the Brexit vote, when Britain voted to leave the European Union.... 05:23 - Source: CNN. London CNN —. Seven months since he announced his resignation as prime minister of the United Kingdom, Boris Johnson's shadow still looms large over the ruling ... Boris Johnson (born June 19, 1964, New York City, New York, U.S.) American-born British journalist and Conservative Party politician who became prime minister of the Unit

D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


The current prime minister of the United Kingdom is Rishi Sunak, who took office in October 2022. This web page provides a comprehensive list of all the prime ministers of the UK since 1721, with their portraits, terms of office, parties, and governments. Rishi Sunak (born May 12, 1980, Southampton, England) British politician and financier who became leader of the Conservative Party and prime minster of the United Kingdom in October 2022. Previously he served as chancellor of the Exchequer (2020-22). Early life Sunak was born into a family with immigrant roots. Boris Johnson (born June 19, 1964, New York City, New York, U.S.) American-born British journalist and Conservative Party politician who became prime minister of the United Kingdom in July 2019. He left office in September 2022 after being forced by scandal to resign. This is a list of current heads of state and heads of government. In some cases, mainly in presidential systems, there is only one leader being both head of state

D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [13]:
print(output['input'])


Answer the following questions as best uou can.
Questions: Who is the current prime minister of the UK?



In [14]:
output['output']

'The current prime minister of the UK is Rishi Sunak.'

In [18]:
question = 'Tell me about Napolean bonaparte?'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...
I need to find information about Napoleon Bonaparte in German.
Action: Action [DuckDuckGo Search]
Action Input: Napoleon Bonaparte German WikipediaAction [DuckDuckGo Search] is not a valid tool, try one of [Python REPL, Wikipedia, DuckDuckGo Search].I should try using Wikipedia to find information about Napoleon Bonaparte in German.
Action: Wikipedia
Action Input: Napoleon Bonaparte GermanPage: House of Bonaparte
Summary: The House of Bonaparte is a former imperial and royal European dynasty of Corsican origin. It was founded in 1804 by Napoleon I, the son of Corsican nobleman Carlo Buonaparte and Letizia Buonaparte (née Ramolino). Napoleon was a French military leader who rose to power during the French Revolution and who, in 1804, transformed the First French Republic into the First French Empire, five years after his coup d'état of November 1799 (18 Brumaire). Napoleon and the Grande Armée had to fight against every major European power (excep

# Vector Databases

In [ ]:
# Pinecone is a vector database that is designed for storing a query of high dimensional
# vectors. It provides fast and efficient semantic search over vector embeddings by integrating open ai
# open ai LLMs, we fine deep learning capability for embedding generation with efficient vector search
# storage and retrieval .

# Vector databases are a new type of database, designed to store and query unstructured data.
# Unstructured data is data that does not have a fixed schema, such as text, images and audio.

# Pipeline for Vector Databases
# Vector databases use a combination of different optimized algorithms that all participate in
# Approximate Nearest Neighbor (ANN) search.

# Steps 1> Embedding, 2> Indexing, 3> Querying




In [19]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [20]:
pip install -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install --upgrade -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip show pinecone-client

Name: pinecone-client
Version: 3.0.3
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [27]:
from pinecone import Pinecone
pc = Pinecone()
#pc = Pinecone(api_key='d5fa5468-8b37-4add-a187-42f0e7c3c675')
pc.list_indexes()

{'indexes': []}